In [ ]:
import numpy as np
from tqdm import tqdm
from pathlib import Path
import gtsam
from gtsam.symbol_shorthand import X, L
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from lac.slam.gtsam_factor_graph import GtsamFactorGraph
from lac.slam.gtsam_util import remove_outliers, plot_reprojection_residuals
from lac.slam.slam import SLAM, K
from lac.slam.visual_odometry import StereoVisualOdometry
from lac.slam.feature_tracker import FeatureTracker
from lac.utils.plotting import plot_poses, plot_surface, plot_3d_points, plot_path_3d
from lac.utils.visualization import image_grid
from lac.util import load_data, load_stereo_images, load_images

%load_ext autoreload
%autoreload 2

In [ ]:
# Load the data logs
data_path = "/home/shared/data_raw/LAC/runs/full_spiral_map1_preset0"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)
print(f"Loaded {len(poses)} poses")

In [ ]:
left_imgs, right_imgs = load_stereo_images(data_path, start_frame=0, end_frame=10000)
images = {"FrontLeft": left_imgs, "FrontRight": right_imgs}

# Left and Right projection factors


In [ ]:
from lac.utils.frames import get_cam_pose_rover, CAMERA_TO_OPENCV_PASSIVE

In [ ]:
pixel_noise = gtsam.noiseModel.Isotropic.Sigma(2, 2.0)
huber = gtsam.noiseModel.mEstimator.Huber(k=1.345)
robust_pixel_noise = gtsam.noiseModel.Robust.Create(huber, gtsam.noiseModel.Isotropic.Sigma(2, 1.0))

In [ ]:
rover_T_cam_FL = get_cam_pose_rover("FrontLeft")
rover_T_cam_FL[:3, :3] = rover_T_cam_FL[:3, :3] @ CAMERA_TO_OPENCV_PASSIVE
ROVER_T_CAM_FRONT_LEFT = gtsam.Pose3(rover_T_cam_FL)
rover_T_cam_FR = get_cam_pose_rover("FrontRight")
rover_T_cam_FR[:3, :3] = rover_T_cam_FR[:3, :3] @ CAMERA_TO_OPENCV_PASSIVE
ROVER_T_CAM_FRONT_RIGHT = gtsam.Pose3(rover_T_cam_FR)

In [ ]:
START_FRAME = 80
END_FRAME = 4000

FIX_FIRST_POSE = False
ADD_RIGHT_FACTORS = True
USE_ROBUST_NOISE = True

In [ ]:
# Initialize modules
tracker = FeatureTracker(cam_config)
tracker.initialize(poses[START_FRAME], left_imgs[START_FRAME], right_imgs[START_FRAME])

svo = StereoVisualOdometry(cam_config)
svo.initialize(poses[START_FRAME], left_imgs[START_FRAME], right_imgs[START_FRAME])

graph = gtsam.NonlinearFactorGraph()
initial_estimate = gtsam.Values()

# Add first pose
if FIX_FIRST_POSE:
    graph.add(gtsam.NonlinearEqualityPose3(X(0), gtsam.Pose3(poses[START_FRAME])))
else:
    initial_pose_noise = gtsam.noiseModel.Isotropic.Sigma(6, 1e-3)
    graph.add(gtsam.PriorFactorPose3(X(0), gtsam.Pose3(poses[START_FRAME]), initial_pose_noise))
initial_estimate.insert(X(0), gtsam.Pose3(poses[START_FRAME]))

landmark_ids = set()
curr_pose = poses[START_FRAME]
svo_poses = [poses[START_FRAME]]

# Add first landmarks
for i, id in enumerate(tracker.track_ids):
    if id not in landmark_ids:
        landmark_ids.add(id)
        initial_estimate.insert(L(id), tracker.world_points[i])
    graph.add(
        gtsam.GenericProjectionFactorCal3_S2(
            tracker.prev_pts[i],
            robust_pixel_noise,
            X(0),
            L(id),
            K,
            ROVER_T_CAM_FRONT_LEFT,
        )
    )
    if ADD_RIGHT_FACTORS:
        graph.add(
            gtsam.GenericProjectionFactorCal3_S2(
                tracker.prev_pts_right[i],
                robust_pixel_noise,
                X(0),
                L(id),
                K,
                ROVER_T_CAM_FRONT_RIGHT,
            )
        )

In [ ]:
pose_key = 1

for frame in tqdm(range(START_FRAME + 2, END_FRAME, 2)):
    # VO
    svo.track(images["FrontLeft"][frame], images["FrontRight"][frame])
    curr_pose = svo.get_pose()
    svo_poses.append(curr_pose)

    # Feature tracking
    tracker.track_keyframe(curr_pose, images["FrontLeft"][frame], images["FrontRight"][frame])

    # Update the graph
    initial_estimate.insert(X(pose_key), gtsam.Pose3(curr_pose))

    # Add vision factors
    for i, id in enumerate(tracker.track_ids):
        if id not in landmark_ids:
            landmark_ids.add(id)
            initial_estimate.insert(L(id), tracker.world_points[i])
        graph.add(
            gtsam.GenericProjectionFactorCal3_S2(
                tracker.prev_pts[i],
                robust_pixel_noise,
                X(pose_key),
                L(id),
                K,
                ROVER_T_CAM_FRONT_LEFT,
            )
        )
        if ADD_RIGHT_FACTORS:
            graph.add(
                gtsam.GenericProjectionFactorCal3_S2(
                    tracker.prev_pts_right[i],
                    robust_pixel_noise,
                    X(pose_key),
                    L(id),
                    K,
                    ROVER_T_CAM_FRONT_RIGHT,
                )
            )

    pose_key += 1

In [ ]:
print(f"Initial error: {graph.error(initial_estimate)}")
params = gtsam.LevenbergMarquardtParams()
params.setVerbosity("TERMINATION")
optimizer = gtsam.LevenbergMarquardtOptimizer(graph, initial_estimate, params)
result = optimizer.optimize()
print(f"Final error: {graph.error(result)}")

In [ ]:
print(f"Initial error: {graph.error(initial_estimate)}")
params = gtsam.GncLMParams()
params.setVerbosityGNC(gtsam.GncLMParams.Verbosity.SUMMARY)
optimizer = gtsam.GncLMOptimizer(graph, initial_estimate, params)
result = optimizer.optimize()
print(f"Final error: {graph.error(result)}")

In [ ]:
gnc_result = result

In [ ]:
est_traj = np.array([result.atPose3(X(k)).translation() for k in range(pose_key)])
fig = plot_poses(poses[:END_FRAME], no_axes=True, color="black", name="Ground Truth")
fig = plot_poses(svo_poses, fig=fig, no_axes=True, color="blue", name="VO")
fig = plot_path_3d(est_traj, fig=fig, color="red", name="GTSAM")
fig.show()

In [ ]:
fig.write_html("../../../results/slam/gnc_opt.html")

In [ ]:
plot_reprojection_residuals(graph, result)

In [ ]:
cleaned_graph, cleaned_estimate = remove_outliers(graph, result)

In [ ]:
print(f"Initial error: {cleaned_graph.error(cleaned_estimate)}")
params = gtsam.GncLMParams()
params.setVerbosityGNC(gtsam.GncLMParams.Verbosity.SUMMARY)
optimizer = gtsam.GncLMOptimizer(cleaned_graph, cleaned_estimate, params)
result = optimizer.optimize()
print(f"Final error: {cleaned_graph.error(result)}")

In [ ]:
print(f"Initial error: {cleaned_graph.error(cleaned_estimate)}")
params = gtsam.LevenbergMarquardtParams()
params.setVerbosity("TERMINATION")
# params.setVerbosity("ERROR")
optimizer = gtsam.LevenbergMarquardtOptimizer(cleaned_graph, cleaned_estimate, params)
result = optimizer.optimize()
print(f"Final error: {cleaned_graph.error(result)}")

In [ ]:
est_traj = np.array([result.atPose3(X(k)).translation() for k in range(1959)])

fig = plot_poses(poses[:END_FRAME], no_axes=True, color="black", name="Ground Truth")
fig = plot_poses(svo_poses, fig=fig, no_axes=True, color="blue", name="VO")
fig = plot_path_3d(est_traj, fig=fig, color="red", name="GTSAM")
fig.show()

In [ ]:
plot_reprojection_residuals(cleaned_graph, result)

In [ ]:
cleaned_graph, cleaned_estimate = remove_outliers(cleaned_graph, result)

In [ ]:
print(f"Initial error: {cleaned_graph.error(cleaned_estimate)}")
params = gtsam.LevenbergMarquardtParams()
params.setVerbosity("TERMINATION")
# params.setVerbosity("ERROR")
optimizer = gtsam.LevenbergMarquardtOptimizer(cleaned_graph, cleaned_estimate, params)
result = optimizer.optimize()
print(f"Final error: {cleaned_graph.error(result)}")

In [ ]:
est_traj = np.array([result.atPose3(X(k)).translation() for k in range(1959)])

fig = plot_poses(poses[:END_FRAME], no_axes=True, color="black", name="Ground Truth")
fig = plot_poses(svo_poses, fig=fig, no_axes=True, color="blue", name="VO")
fig = plot_path_3d(est_traj, fig=fig, color="red", name="GTSAM")
fig.show()

In [ ]:
plot_reprojection_residuals(cleaned_graph, result)